# Taller 10 - Twitter

In [201]:
import pandas as pd
import tweepy

import re
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

In [178]:
keys = pd.read_csv("C:/Users/Nadia/Documents/Especialización/NLP/Clases y Talleres/Claves_NLP_Twitter.txt", header=None)


In [181]:
# Autorizar a través API
logueo = tweepy.OAuthHandler(keys['api_key'], keys['api_key_secret'])
logueo.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(logueo, wait_on_rate_limit=True)

Recolectar Información

In [182]:
tweets = tweepy.Cursor(api.search,
                       q=['netflix -RT'],
                       lang='es',
                       geocode=f'4.649178,-74.062827,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(300)

In [183]:
for t in tweets:
    print (t)
    break

Status(_api=<tweepy.api.API object at 0x000001FE8D98DCC8>, _json={'created_at': 'Sat Nov 14 12:07:50 +0000 2020', 'id': 1327584021946175489, 'id_str': '1327584021946175489', 'full_text': 'Tío Netflix esta sobreactuando con las películas de navidad', 'truncated': False, 'display_text_range': [0, 59], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'es', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 794500592785825793, 'id_str': '794500592785825793', 'name': 'Erika Prada', 'screen_name': 'Eri_pl', 'location': 'Pereira, Colombia', 'description': 'Volví, y ahora que hago?', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 75, 'frien

In [184]:
t._json

{'created_at': 'Sat Nov 14 12:07:50 +0000 2020',
 'id': 1327584021946175489,
 'id_str': '1327584021946175489',
 'full_text': 'Tío Netflix esta sobreactuando con las películas de navidad',
 'truncated': False,
 'display_text_range': [0, 59],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'metadata': {'iso_language_code': 'es', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 794500592785825793,
  'id_str': '794500592785825793',
  'name': 'Erika Prada',
  'screen_name': 'Eri_pl',
  'location': 'Pereira, Colombia',
  'description': 'Volví, y ahora que hago?',
  'url': None,
  'entities': {'description': {'urls': []}},
  'protected': False,
  'followers_count': 75,
  'friends_count': 102,
  'listed_count

In [185]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogotá")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)
    

In [186]:
datos = pd.DataFrame(t)

In [187]:
datos.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
datos.head()

,tweet,date,city,user_location,id,link
0,"@lagordaniembro La mayoría de las que son ""Net...",2020-11-14 11:33:11,Bogotá,Bogotá,1327575302344728577,https://twitter.com/i/web/status/1327575302344...
1,"@TheWereDrunk La serie está en Netflix, perten...",2020-11-14 11:11:43,Bogotá,"Bogotá, Colombia",1327569897015144449,https://twitter.com/i/web/status/1327569897015...
2,La verdá les pido que la vean para que Netflix...,2020-11-14 07:08:04,Bogotá,"Bogotá, Colombia",1327508582716350464,https://twitter.com/i/web/status/1327508582716...
3,Tienen que verse Gambito de Dama en Netflix. E...,2020-11-14 06:12:22,Bogotá,"Manizales, Caldas",1327494564949745664,https://twitter.com/i/web/status/1327494564949...
4,Estoy a esto 👌 de volverme vegana. Luego de ve...,2020-11-14 05:34:52,Bogotá,Bogotá Colombia,1327485128046743552,https://twitter.com/i/web/status/1327485128046...


Preprocesamiento para Tweets

In [188]:
datos = datos[['tweet','date','city']]
datos.head()

,tweet,date,city
0,"@lagordaniembro La mayoría de las que son ""Net...",2020-11-14 11:33:11,Bogotá
1,"@TheWereDrunk La serie está en Netflix, perten...",2020-11-14 11:11:43,Bogotá
2,La verdá les pido que la vean para que Netflix...,2020-11-14 07:08:04,Bogotá
3,Tienen que verse Gambito de Dama en Netflix. E...,2020-11-14 06:12:22,Bogotá
4,Estoy a esto 👌 de volverme vegana. Luego de ve...,2020-11-14 05:34:52,Bogotá


In [208]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto


In [209]:

datos['tweet_p'] = datos['tweet'].apply(lambda texto: pre_procesado(texto))

datos.head()

,tweet,date,city,tweet_p
0,"@lagordaniembro La mayoría de las que son ""Net...",2020-11-14 11:33:11,Bogotá,"[lagordaniembro, mayoría, netflix, original]"
1,"@TheWereDrunk La serie está en Netflix, perten...",2020-11-14 11:11:43,Bogotá,"[theweredrunk, serie, netflix, pertenece, catá..."
2,La verdá les pido que la vean para que Netflix...,2020-11-14 07:08:04,Bogotá,"[verdá, pido, vean, netflix, cancele, tener, q..."
3,Tienen que verse Gambito de Dama en Netflix. E...,2020-11-14 06:12:22,Bogotá,"[verse, gambito, dama, netflix, buenísima]"
4,Estoy a esto 👌 de volverme vegana. Luego de ve...,2020-11-14 05:34:52,Bogotá,"[volverme, vegana, luego, ver, what, the, heal..."


In [210]:
datos.tweet_p.iloc[1]

['theweredrunk',
 'serie',
 'netflix',
 'pertenece',
 'catálogo',
 'series',
 'televisión',
 'española']

In [156]:
datos.tweet.iloc[1]

'La verdá les pido que la vean para que Netflix no la cancele y tener con quién hablar de ella.'

In [211]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet_p']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [ ]:
datos[['tweet_clean', 'links_2', 'hashtags', 'mentions']] = datos.apply(text_clean, axis=1, result_type='expand')

In [138]:
datos.head()

,tweet,date,city,tweet_p
0,"@TheWereDrunk La serie está en Netflix, perten...",2020-11-14 11:11:43,Bogotá,"[theweredrunk, serie, netflix, pertenece, catá..."
1,La verdá les pido que la vean para que Netflix...,2020-11-14 07:08:04,Bogotá,"[verdá, pido, vean, netflix, cancele, tener, q..."
2,Tienen que verse Gambito de Dama en Netflix. E...,2020-11-14 06:12:22,Bogotá,"[verse, gambito, dama, netflix, buenísima]"
3,Estoy a esto 👌 de volverme vegana. Luego de ve...,2020-11-14 05:34:52,Bogotá,"[volverme, vegana, luego, ver, what, the, heal..."
4,@vamarogu Huy si la locura 📺 Netflix 🍟🍔y ya,2020-11-14 05:32:53,Bogotá,"[vamarogu, huy, si, locura, netflix]"


In [214]:
pip install regex


Note: you may need to restart the kernel to use updated packages.


In [215]:
pip install emoji

  Created wheel for emoji: filename=emoji-0.6.0-cp37-none-any.whl size=49720 sha256=5cc97c4c30ba42a58d99a6e80113cd83ccb9e4fbf8611ae74a8c999e35a6391c
  Stored in directory: C:\Users\Nadia\AppData\Local\pip\Cache\wheels\46\2c\8b\9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    datos = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)


    return emoji_list

 
datps['emojis'] = datos['tweet_p'].apply(lambda text: get_emojis(text))